In [54]:
import pandas as pd
import numpy as np
import boto3
import joblib

In [36]:
hit_df = pd.read_csv("../results/results_clean.csv")

In [37]:
# Keep only the most recent pilot
hit_df = hit_df[hit_df['source_x'].str.startswith("pilot03")].copy()

In [38]:
# Real quick, parse the offer_amt
hit_df['offer_amt_str'] = hit_df['offer_amt'].str.replace("$","",regex=False)
hit_df['offer_amt_num'] = pd.to_numeric(hit_df['offer_amt_str'])
hit_df['offer_amt_num'].mean()

1.003763440860215

In [39]:
hit_df['accepted_offer'].value_counts()

True     61
False    32
Name: accepted_offer, dtype: int64

In [40]:
accept_df = hit_df[hit_df['accepted_offer']]
print(len(accept_df))
decline_df = hit_df[~hit_df['accepted_offer']]
print(len(decline_df))

61
32


In [41]:
accept_df['offer_amt_num'].mean()

0.9988524590163934

In [42]:
decline_df['offer_amt_num'].mean()

1.013125

In [43]:
# Keep only people who accepted the offer
hit_df = hit_df[hit_df['accepted_offer']].copy()

In [44]:
hit_df['offer_amt_num']

18     1.50
19     0.50
20     0.50
21     0.99
22     1.01
       ... 
103    0.90
104    1.10
106    1.01
108    0.99
109    0.90
Name: offer_amt_num, Length: 61, dtype: float64

A1E28G16P97EGB 1.5
A2OM3QDLA0IE6U 0.5
A3N17G54NOCPWX 0.5
AJL5DQKFAQ6VW 0.99
AQGWZNJI8VWMK 1.01
A17P7PVFSHRZ3X 1.02
A2196YL67CLX7V 0.95
A3Q8JWXRXCEEEY 1.0
A2ME5SRDS5XTKK 0.5
A3CME8E2E2R7Q7 0.5
ATKD2VNN90SZS 1.01
ALVKFFGYA8N96 1.05
A1607HOEF37WK7 1.01
A2YQ4MH1KOZEIP 1.5
A1Z2A5CGJSEOTN 1.5
A1AJWPPONRMBDO 1.01
AV6J56XKCIG7F 1.01
A1ISANHUO7XYC8 0.98
A129GA7DAJESE2 1.05
A1PAKREEY798TM 0.99
A1AYYY3LHH7ZI4 0.9
A3T7Y10HHQ1VLL 0.9
AHWF0QIK9VTDS 1.0
A3BMJBNAAPO1RN 0.98
A78MLN7DSRJDL 0.95
AKQD9TR2U3H57 1.02
A3QFZDC2ZIWRC 0.99
A31A2PFANDTFQ5 1.05
A3E01SOXXDM9HQ 1.02
A23UEY31KCVDWT 1.0
A1K21ESUNOG8W2 1.01
AP6UQL199WJY 1.5
A2BV8V2M7GYNFZ 1.5
A154KA5M9XPDG4 1.0
A1KJQQ5B4Z9RQU 1.1
A14HPPOF9NSYWD 0.9
A1NWCOQ4IT581P 0.9
ASCZFRBLTOTN4 1.05
A4G7W4HZMDWNP 0.5
A1QSK4SFBARPN 0.95
A3C08PHHSZ60JB 1.02
ADGFXF5EXWGBM 1.5
A1IZZCRVL2RCAM 1.1
A1CDG3PBNGLXKV 1.05
ANWP70ULJ31H1 0.98
ADJ6HSU1QWJKA 1.02
A3KMV3W2JAVRPD 0.98
A768RIEZR3MEY 1.02
A3MTILHYDXWYGI 1.02
AO1USE6LM4M8M 1.02
AQLGKWA76S8RI 0.9
A2193PVPAVJPQO 0.99
AB

In [46]:
hit_df['AssignmentId']

18     31QNSG6A5SUB67IWCL9Q49R122N87H
19     323Q6SJS8JHHEBYVX8185EXH39WFH7
20     323Q6SJS8JHHEBYVX8185EXH39WHF9
21     324G5B4FB49TOIDTHHWBBULJJ5C70W
22     324G5B4FB49TOIDTHHWBBULJJ5G700
                    ...              
103    3XIQGXAUMD91L8J4KTLIK88UTCY7XS
104    3YMU66OBIO9NSBUQ8S7QNZXT4OWGH7
106    3Z7EFSHGNAF7XPE0BATT0U08UMNCXM
108    3Z9WI9EOZ0PSUXQ9YJS5LRPMB5DHKB
109    3ZPBJO59KQ2KGRKZ626IHEUWRWGHDY
Name: AssignmentId, Length: 61, dtype: object

## MTurk API

Parameters

        WorkerId (string) --

        [REQUIRED]

        The ID of the Worker being paid the bonus.
        BonusAmount (string) --

        [REQUIRED]

        The Bonus amount is a US Dollar amount specified using a string (for example, "5" represents $5.00 USD and "101.42" represents $101.42 USD). Do not include currency symbols or currency codes.
        AssignmentId (string) --

        [REQUIRED]

        The ID of the assignment for which this bonus is paid.
        Reason (string) --

        [REQUIRED]

        A message that explains the reason for the bonus payment. The Worker receiving the bonus can see this message.
        UniqueRequestToken (string) -- A unique identifier for this request, which allows you to retry the call on error without granting multiple bonuses. This is useful in cases such as network timeouts where it is unclear whether or not the call succeeded on the server. If the bonus already exists in the system from a previous call using the same UniqueRequestToken, subsequent calls will return an error with a message containing the request ID.

Return type

    dict


In [28]:
# Load access+secret keys
api_keys = pd.read_csv("../../IndependentStudyWithJeff/aws/accessKeys.csv")
key_entry = api_keys.iloc[0]
access_id = key_entry['Access key ID']
secret_id = key_entry['Secret access key']

In [29]:
session = boto3.Session()
client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url="https://mturk-requester.us-east-1.amazonaws.com",
    aws_access_key_id=access_id,
    aws_secret_access_key=secret_id,
)

In [30]:
print(client.get_account_balance()['AvailableBalance'])

499911.45


In [51]:
# Trying first one manually
cur_worker_id = hit_df.iloc[0]['worker_id']
cur_offer_str = hit_df.iloc[0]['offer_amt_str']
cur_assignment_id = hit_df.iloc[0]['AssignmentId']
print(cur_worker_id, cur_offer_str, cur_assignment_id)

A1E28G16P97EGB 1.50 31QNSG6A5SUB67IWCL9Q49R122N87H


In [52]:
# Template:
response = client.send_bonus(
    WorkerId=cur_worker_id,
    BonusAmount=cur_offer_str,
    AssignmentId=cur_assignment_id,
    Reason=reason_text,
    #UniqueRequestToken='string'
)

In [55]:
joblib.dump(response, "bonus_response_{cur_worker_id}_2021-08-30.pkl")

['bonus_response_{cur_worker_id}_2021-08-30.pkl']

Assignment: 31QNSG6A5SUB67IWCL9Q49R122N87H

Ok, now pay the rest automatically

In [61]:
hit_df.iloc[20:]

,HITId,HITTypeId,Title,Description,Reward,CreationTime,MaxAssignments,Expiration,AssignmentId,worker_id,...,response25,response26,response27,response28,response29,response30,source_y,offer_amt,offer_amt_str,offer_amt_num
43,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2021,100,Sat Sep 04 19:33:08 PDT 2021,3E1QT0TDFQA8VRZSVCDNVYHAGIY8IC,A1AYYY3LHH7ZI4,...,Stay at your job without asking for a raise,NaN,NaN,NaN,NaN,NaN,pilot17_Workplace_Survey_Pilot_87_3.csv,$0.90,0.90,0.90
46,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2021,100,Sat Sep 04 19:33:08 PDT 2021,3F1567XTNX6LQU6LDCOA4W37RYXQ9M,A3T7Y10HHQ1VLL,...,Stay at your job without asking for a raise,NaN,NaN,NaN,NaN,NaN,pilot17_Workplace_Survey_Pilot_87_3.csv,$0.90,0.90,0.90
47,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2021,100,Sat Sep 04 19:33:08 PDT 2021,3FFJ6VRIL2PQD2UP08082JBSXKM0IH,AHWF0QIK9VTDS,...,Stay at your job without asking for a raise,NaN,NaN,NaN,NaN,NaN,pilot17_Workplace_Survey_Pilot_87_3.csv,$1.00,1.00,1.00
48,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2021,100,Sat Sep 04 19:33:08 PDT 2021,3FFJ6VRIL2PQD2UP08082JBSXKMI0Z,A3BMJBNAAPO1RN,...,Accept the offer and leave your job,NaN,NaN,NaN,NaN,NaN,pilot17_Workplace_Survey_Pilot_87_3.csv,$0.98,0.98,0.98
50,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2021,100,Sat Sep 04 19:33:08 PDT 2021,3G2UL9A02EFO2TZ8J6CSKVVAW7X67T,A78MLN7DSRJDL,...,Accept the offer and leave your job,NaN,NaN,NaN,NaN,NaN,pilot17_Workplace_Survey_Pilot_87_3.csv,$0.95,0.95,0.95
54,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2021,100,Sat Sep 04 19:33:08 PDT 2021,3IO1LGZLKAYS274RTB4YYVR57R386I,AKQD9TR2U3H57,...,Accept the offer and leave your job,NaN,NaN,NaN,NaN,NaN,pilot17_Workplace_Survey_Pilot_87_3.csv,$1.02,1.02,1.02
55,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2021,100,Sat Sep 04 19:33:08 PDT 2021,3IXQG4FA2UZYMO4WUURK97885YWB9V,A3QFZDC2ZIWRC,...,Accept the offer and leave your job,NaN,NaN,NaN,NaN,NaN,pilot17_Workplace_Survey_Pilot_87_3.csv,$0.99,0.99,0.99
59,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2021,100,Sat Sep 04 19:33:08 PDT 2021,3K5TEWLKGWC663V535OXNC1IRGZIVR,A31A2PFANDTFQ5,...,Stay at your job without asking for a raise,NaN,NaN,NaN,NaN,NaN,pilot17_Workplace_Survey_Pilot_87_3.csv,$1.05,1.05,1.05
60,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2021,100,Sat Sep 04 19:33:08 PDT 2021,3K772S5NP9CP8XYZ2XH2NWCHR8HHER,A3E01SOXXDM9HQ,...,Accept the offer and leave your job,NaN,NaN,NaN,NaN,NaN,pilot17_Workplace_Survey_Pilot_87_3.csv,$1.02,1.02,1.02
61,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2

In [58]:
hit_df[hit_df['worker_id'] == "A3BMJBNAAPO1RN"]

,HITId,HITTypeId,Title,Description,Reward,CreationTime,MaxAssignments,Expiration,AssignmentId,worker_id,...,response25,response26,response27,response28,response29,response30,source_y,offer_amt,offer_amt_str,offer_amt_num
48,3O71U79SRCQ9NA6Y4NCIGYS2DMOMSP,33C7SA8UDA0HD50TAVWCSIAXO6TOSL,Quick 3-question survey about work [<15 second...,Quick <15 second survey about workplace condit...,$0.15,Sat Aug 28 19:33:08 PDT 2021,100,Sat Sep 04 19:33:08 PDT 2021,3FFJ6VRIL2PQD2UP08082JBSXKMI0Z,A3BMJBNAAPO1RN,...,Accept the offer and leave your job,NaN,NaN,NaN,NaN,NaN,pilot17_Workplace_Survey_Pilot_87_3.csv,$0.98,0.98,0.98


In [62]:
reason_text = "Additional survey questions on the workplace survey"
for row_index, cur_row in hit_df.iloc[1:].iterrows():
    cur_worker_id = cur_row['worker_id']
    cur_offer_str = cur_row['offer_amt_str']
    cur_assignment_id = cur_row['AssignmentId']
    if row_index <= 48:
        # We already paid these
        continue
    print(f"Paying {cur_worker_id} bonus of {cur_offer_str} for assignment {cur_assignment_id}")
    # Template:
    response = client.send_bonus(
        WorkerId=cur_worker_id,
        BonusAmount=cur_offer_str,
        AssignmentId=cur_assignment_id,
        Reason=reason_text,
        #UniqueRequestToken='string'
    )
    print(response)
    responses.append(response)

Paying A78MLN7DSRJDL bonus of 0.95 for assignment 3G2UL9A02EFO2TZ8J6CSKVVAW7X67T
{'ResponseMetadata': {'RequestId': '5106a7a3-370c-4a3b-ae00-796e54b8fdbe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5106a7a3-370c-4a3b-ae00-796e54b8fdbe', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Mon, 30 Aug 2021 22:37:20 GMT'}, 'RetryAttempts': 0}}
Paying AKQD9TR2U3H57 bonus of 1.02 for assignment 3IO1LGZLKAYS274RTB4YYVR57R386I
{'ResponseMetadata': {'RequestId': 'b26c69b6-75c0-4ef5-a961-ba9eff93d1f8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b26c69b6-75c0-4ef5-a961-ba9eff93d1f8', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Mon, 30 Aug 2021 22:37:20 GMT'}, 'RetryAttempts': 0}}
Paying A3QFZDC2ZIWRC bonus of 0.99 for assignment 3IXQG4FA2UZYMO4WUURK97885YWB9V
{'ResponseMetadata': {'RequestId': 'ad849dfe-503b-4006-bd18-c362069d8b8a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ad849dfe-5

In [63]:
joblib.dump(responses, "bonus_responses_pilot03_2021-08-30.pkl")

['bonus_responses_pilot03_2021-08-30.pkl']

## And now assign qualifications

In [64]:
# Trying with first worker first
cur_worker_id = hit_df.iloc[0]['worker_id']
print(cur_worker_id)

A1E28G16P97EGB


In [65]:
survey_qual_id = "303SJT1CWE5D23ZEJCT5ZGDLM1P4FN"

In [66]:
# Template
response = client.associate_qualification_with_worker(
    QualificationTypeId=survey_qual_id,
    WorkerId=cur_worker_id,
    IntegerValue=15,
    SendNotification=False
)

In [68]:
response0 = response

Assign the rest automatically

In [69]:
responses = []
responses.append(response0)
for row_index, cur_row in hit_df.iloc[1:].iterrows():
    cur_worker_id = cur_row['worker_id']
    print(f"Assigning qualification to worker {cur_worker_id}")
    response = client.associate_qualification_with_worker(
        QualificationTypeId=survey_qual_id,
        WorkerId=cur_worker_id,
        IntegerValue=15,
        SendNotification=False
    )
    responses.append(response)

In [70]:
joblib.dump(responses, "qual_responses_pilot03_2021-08-30.pkl")

['qual_responses_pilot03_2021-08-30.pkl']